# Jedi Council sentiments

### "Fear is the path to the dark side... fear leads to anger... anger leads to hate.. hate leads to suffering."

![picture](./images/jedi-council.png)

Vamos a analizar con nuestra API el sentimiento de este famoso diálogo del Episodio I. Importamos brevemente el dataset y nos quedamos con las frases que nos interesan.

In [ ]:
import pandas as pd
yoda = pd.read_csv('inputs/yoda-dialogues.csv')
yoda_filt = yoda[(yoda['character'] != 'narrator') & (yoda['scene']==134)][['character','text']]
yoda_filt

### Ahora insertamos el dataset en MongoDB con llamadas a la API

In [ ]:
import requests

Insertamos a los personajes que participan

In [ ]:
jedis = list(yoda_filt['character'].unique())
for jedi in jedis:
    res = requests.get(f'http://localhost:5000/user/create/{jedi}').text
    print(res)

Creamos un grupo con ellos

In [ ]:
data = {"nombre":"Jedi Council, Ep I",
        "usuarios":jedis}
resultado = requests.get('http://localhost:5000/chat/create', params=data).text
print(resultado)

In [ ]:
chatid = resultado[5:9]

Insertamos los mensajes

In [ ]:
for a, b in zip(yoda_filt['character'], yoda_filt['text']):
    enviado = requests.get(f'http://localhost:5000/chat/{chatid}/addmessage', params={"usuario":a,"mensaje":b}).text
    print(enviado)

Analizamos el sentimiento del diálogo, que como vemos sale más negativo que positivo:

In [ ]:
response = requests.get(f'http://localhost:5000/chat/{chatid}/sentiment').json()
response

From the documentation:
- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
- negative sentiment: compound score <= -0.05

In [ ]:
recom = requests.get('http://localhost:5000/user/YODA/recommend').json()
recom